In [19]:
import cv2 as cv
from matplotlib import pyplot as plt
import numpy as np


In [20]:
def printImage(legend, source):
    cv.imshow(legend, source)
    cv.waitKey(0)
    cv.destroyAllWindows()

In [31]:
image = cv.imread('flower.png')
printImage("imagem",image)

In [37]:
def f(t):
  if t > 0.008856:
    return t**(1/3)
  else:
    return (903.3*t + 16)/116

def L(Y):
  (Xn, Yn, Zn) = (255, 255, 255)
  l = 116*f(Y/Yn) - 16
  return l

def A(X, Y):
  (Xn, Yn, Zn) = (255, 255, 255)
  a = 500 * ( f(X/Xn) - f(Y/Yn))
  return a

def B(Y,Z):
  (Xn, Yn, Zn) = (255, 255, 255)
  b = 200 * (f(Y/Yn) - f(Z/Zn))
  return b
  


def XYZ2LAB(imageXYZ):
  imageLAB = np.zeros(shape = imageXYZ.shape)
  for i in range(imageXYZ.shape[0]):
    for j in range(imageXYZ.shape[1]):
      imageLAB[i][j][0]= L(image[i][j][1]) #Y
      imageLAB[i][j][1] = A(imageXYZ[i][j][0],imageXYZ[i][j][1]) #X, Y
      imageLAB[i][j][2] = B(imageXYZ[i][j][1],imageXYZ[i][j][2]) #Y, Z

  return imageLAB



def distancia_euclid(pixelIm1, pixelIm2):
  a = (pixelIm1[0] - pixelIm2[0])**2
  b = (pixelIm1[2] - pixelIm2[1])**2
  c = (pixelIm1[2] - pixelIm2[2])**2
  soma = a + b + c
  return np.sqrt(soma)

def FTSD(image):
  printImage("RGB",image)

  printImage("XYZ", image)
  image = np.float64(image)
  image = XYZ2LAB(image)
  

  #I_mean
  L, A, B = cv.split(image)
  image_mean = np.zeros(shape = image.shape)
  image_mean[:,:,0] = np.mean(L)
  image_mean[:,:,1] = np.mean(A)
  image_mean[:,:,2] = np.mean(B)

  #I_gaussianBlured
  image_blurred = cv.GaussianBlur(image, (5,5), sigmaX = 1)


  saliencyMap = np.zeros(shape = (image.shape[0], image.shape[1]))

  #S = ||I_mean - I_gaussianBlured||
  for i in range(image.shape[0]):
    for j in range(image.shape[1]):
      saliencyMap[i][j] = distancia_euclid(image_mean[i][j][:], image_blurred[i][j][:])


  #saliencyMap = np.uint8(saliencyMap)
  printImage("imagem",saliencyMap)

  return saliencyMap

saliency = FTSD(image)
#printImage("image", image)



In [40]:
_, threshold = cv.threshold(saliency, 65, 255, type = cv.THRESH_BINARY)
printImage("imagem",threshold)